# Bottleneck Floy-Warshall

In [ ]:
import pandas as pd
import numpy as np
import scipy.spatial
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()

## Load data

### Option 1: Generate points

In [ ]:
NUM_POINTS = 500
NUM_FEATURES = 12

points = np.random.rand(NUM_POINTS, NUM_FEATURES)

### Option 2: Load points from CSV

In [ ]:
df = pd.read_csv("features.csv")
points = np.array(df)

## Compute pairwise distances

In [ ]:
NORM_P = 0.5

def pairwise_distances(x: np.ndarray):
    """
    Given an array with shape (n_points, n_features), compute the pairwise distance matrix.
    """
    return scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(points, "minkowski", p=NORM_P))

sns.displot(pairwise_distances(points).flatten())

## Compute the best path through the points

In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import floyd_warshall


class BottleneckFloydWarshallTSP:
    def __init__(self, *, points: np.ndarray):
        self.points = points
        self.distance_matrix = pairwise_distances(points)

    def all_pairs_bottleneck_shortest_paths(self):
        graph = csr_matrix(self.points)
        matrix = floyd_warshall(graph, directed=False, return_predecessors=True)
        print(matrix)
        return matrix

    def best_path(self):
        shortest_paths_matrix = self.all_pairs_bottleneck_shortest_paths()

In [ ]:
optimizer = BottleneckFloydWarshallTSP(points=points)
best_path = optimizer.best_path()
best_path